# libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Preprocessing

In [2]:

df_modcloth = pd.read_json("modcloth_final_data.json", lines=True)

In [3]:
df_modcloth_clean = df_modcloth.dropna(subset=["user_id","item_id"]).drop_duplicates()
df_modcloth_clean['category'] = df_modcloth_clean['category'].fillna('unknown')

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

category_bow = vectorizer.fit_transform(df_modcloth_clean['category'])

In [5]:
###############################################
# 3. Implicit Feedback
###############################################

fit_mapping = {'fit': 0, 'large': 2, 'small': 1}
df_modcloth_clean.loc[:, 'fit'] = df_modcloth_clean['fit'].map(fit_mapping)


user_reviews = df_modcloth_clean.groupby('user_id')['review_text'].count().to_dict()
df_modcloth_clean.loc[:, 'num_user_reviews'] = df_modcloth_clean['user_id'].map(user_reviews).fillna(0).astype(int)
max_reviews = df_modcloth_clean['num_user_reviews'].max()
df_modcloth_clean.loc[:, 'num_user_reviews_normalized'] = df_modcloth_clean['num_user_reviews'] / max_reviews

max_quality = df_modcloth_clean['quality'].max()
df_modcloth_clean.loc[:, 'quality_normalized'] = df_modcloth_clean['quality'] / max_quality


df_modcloth_clean.loc[:, 'interaction_strength'] = (
    df_modcloth_clean['fit'] * 0.33 +
    df_modcloth_clean['num_user_reviews'] * 0.33 +
    df_modcloth_clean['quality'] * 0.34
)

In [6]:
df_modcloth_clean['fit'] = df_modcloth_clean['fit'].fillna(df_modcloth_clean['fit'].median())
df_modcloth_clean['num_user_reviews'] = df_modcloth_clean['num_user_reviews'].fillna(df_modcloth_clean['num_user_reviews'].median())
df_modcloth_clean['quality'] = df_modcloth_clean['quality'].fillna(df_modcloth_clean['quality'].median())

<ipython-input-6-939be0d95d69>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_modcloth_clean['fit'] = df_modcloth_clean['fit'].fillna(df_modcloth_clean['fit'].median())


In [7]:
def calculate_sparsity(df):
    """Calculates sparsity for each numerical feature in the DataFrame."""
    numerical_features = df.select_dtypes(include=np.number).columns
    sparsity_results = {}

    for feature in numerical_features:
        sparsity = df[feature].isnull().sum() / len(df)
        sparsity_results[feature] = sparsity

    return sparsity_results

sparsity_dict = calculate_sparsity(df_modcloth_clean)

for feature, sparsity in sparsity_dict.items():
    print(f"Sparsity of {feature}: {sparsity:.2f}")

Sparsity of item_id: 0.00
Sparsity of waist: 0.97
Sparsity of size: 0.00
Sparsity of quality: 0.00
Sparsity of hips: 0.32
Sparsity of bra size: 0.07
Sparsity of fit: 0.00
Sparsity of user_id: 0.00
Sparsity of shoe size: 0.66
Sparsity of num_user_reviews: 0.00
Sparsity of num_user_reviews_normalized: 0.00
Sparsity of quality_normalized: 0.00


In [8]:
def preprocess_data(df, sparsity_threshold=0.50, user_id_col='user_id'):
    """Preprocesses the data by removing numerical features with high sparsity,
       excluding the user identifier column.
    """
    numerical_cols = df.select_dtypes(include=np.number).columns
    cols_to_check = [col for col in numerical_cols if col != user_id_col]

    for col in cols_to_check:
        sparsity = df[col].isnull().sum() / len(df)
        if sparsity > sparsity_threshold:
            print(f"Dropping sparse feature: {col} (Sparsity: {sparsity:.2f})")
            df = df.drop(col, axis=1)
    user_interaction_counts = df_modcloth_clean.groupby('user_id')['interaction_strength'].count()
    print(user_interaction_counts)
    users_with_more_than_two_interactions = user_interaction_counts[user_interaction_counts >= 2]
    print(f"Number of users with more than 2 interactions: {len(users_with_more_than_two_interactions)}")
    df = df_modcloth_clean[df_modcloth_clean["user_id"].isin(users_with_more_than_two_interactions.index)]

    return df

In [9]:
df_modcloth_clean = preprocess_data(df_modcloth_clean, sparsity_threshold=0.50)

Dropping sparse feature: waist (Sparsity: 0.97)
Dropping sparse feature: shoe size (Sparsity: 0.66)
user_id
6         1
46        1
55        1
66        4
104       2
         ..
999864    1
999887    2
999888    1
999923    3
999972    3
Name: interaction_strength, Length: 47958, dtype: int64
Number of users with more than 2 interactions: 15924


In [10]:
df_modcloth_clean.head()

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,...,fit,user_id,shoe size,shoe width,review_summary,review_text,num_user_reviews,num_user_reviews_normalized,quality_normalized,interaction_strength
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,...,1,991571,NaN,NaN,NaN,NaN,2,0.08,1.0,2.69
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,...,1,944840,NaN,NaN,NaN,NaN,1,0.04,1.0,2.36
5,123373,27.0,11,5.0,c,41.0,36.0,new,NaN,5ft 4in,...,1,162012,NaN,NaN,NaN,NaN,1,0.04,1.0,2.36
8,123373,NaN,30,4.0,d,50.0,42.0,new,NaN,5ft 10in,...,1,279568,11.0,wide,NaN,NaN,4,0.16,0.8,3.01
9,123373,NaN,13,5.0,dd/e,41.0,36.0,new,39,5ft 6in,...,0,950172,9.0,NaN,NaN,NaN,2,0.08,1.0,2.36


In [11]:
user_activity = df_modcloth_clean.groupby('user_id')['interaction_strength'].sum()

In [12]:
median_interaction_strength = df_modcloth_clean.groupby('user_id')['interaction_strength'].sum().median()

df_modcloth_clean['user_activity'] = df_modcloth_clean.groupby('user_id')['interaction_strength'].transform('sum').map(
    lambda x: 'Low' if x < median_interaction_strength else 'High'
)

print(df_modcloth_clean['user_activity'].value_counts())

user_activity
High    34900
Low     15471
Name: count, dtype: int64


# SPLIT AND MATRIX

In [13]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df_modcloth_clean, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [14]:
###############################################
# 3. User Item Matrix
###############################################

# Rows = users.
#Columns = items.
#cell values represent user  interacted with an item.

train_user_item_matrix = train_df.pivot_table(
    index="user_id",
    columns="item_id",
    values="interaction_strength",
    aggfunc="sum",
    fill_value=0
)
val_user_item_matrix = val_df.pivot_table(
    index="user_id",
    columns="item_id",
    values="interaction_strength",
    aggfunc="sum",
    fill_value=0
)
test_user_item_matrix = test_df.pivot_table(
    index="user_id",
    columns="item_id",
    values="interaction_strength",
    aggfunc="sum",
    fill_value=0
)

<ipython-input-14-f947f7bf47dd>:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_user_item_matrix = train_df.pivot_table(
<ipython-input-14-f947f7bf47dd>:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_user_item_matrix = val_df.pivot_table(
<ipython-input-14-f947f7bf47dd>:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_

# Definition ndcg, hr,mrr

In [15]:
###############################################
# 3. Utility/Ranking metrics
###############################################
def dcg_at_k_recursive(relevance_scores, k, b=2):
    """
    Calculate the Discounted Cumulative Gain at rank k recursively.

    :param relevance_scores: A list or array of relevance scores for the ranked items.
    :param k: The rank at which to stop (k items to evaluate).
    :param b: The base of the logarithm (typically 2).
    :return: DCG at rank k.
    """
    k = min(k, len(relevance_scores))
    if k == 0:
        return 0.0

    if k < b:
        return np.sum([rel / np.log2(idx + 2) for idx, rel in enumerate(relevance_scores[:k])])

    else:
        dcg_k_minus_1 = dcg_at_k_recursive(relevance_scores, k - 1, b)
        rel_k = relevance_scores[k - 1]
        return dcg_k_minus_1 + (rel_k / np.log2(k + 1))  #

def dcg_at_k(relevance_scores, k, b=2):
    """
    Calculate the Discounted Cumulative Gain at rank k (non-recursive).
    """
    dcg = 0
    for idx, rel in enumerate(relevance_scores[:k]):
        dcg += rel / np.log2(idx + 2)
    return dcg
def idcg_at_k(relevance_scores, k):
    """
    Calculate the Ideal Discounted Cumulative Gain at rank k.
    :param relevance_scores: A list or array of relevance scores for the ranked items.
    :param k: The rank at which to stop (k items to evaluate).
    :return: Ideal DCG at rank k.
    """
    relevance_scores = sorted(relevance_scores, reverse=True)
    return dcg_at_k(relevance_scores, k)

def ndcg_at_k(relevance_scores, k):
    """
    Calculate the Normalized Discounted Cumulative Gain at rank k.
    :param relevance_scores: A list or array of relevance scores for the ranked items.
    :param k: The rank at which to stop (k items to evaluate).
    :return: nDCG at rank k.
    """
    dcg = dcg_at_k_recursive(relevance_scores, k)
    idcg = idcg_at_k(relevance_scores, k)
    if idcg == 0:
        return 0
    return dcg / idcg


In [16]:
def hr_at_k(actual, predicted, k=5):
    """
    Computes Hit Rate (HR) at rank k.

    Args:
        actual: list of relevant items
        predicted: ranked list of items
        k: rank cutoff (default is 5)

    Returns:
        Hit Rate at rank k.
    """

    for item in predicted[:k]:
        if item in actual:
            return 1
    return 0



def hit_rate_at_k(actual, predicted, k=5):
    """
    Computes Mean Hit Rate (HR) at rank k for all users.

    Args:
        actual: list of relevant items for each user
        predicted: list of predicted ranked items for each user
        k: rank cutoff (default is 5)

    Returns:
        HR at rank k.
    """

    if not actual or len(predicted) == 0:
        return 0.0

    hr_scores = []
    for user_actual, user_predicted in zip(actual, predicted):
        hr = hr_at_k(user_actual, user_predicted, k)
        hr_scores.append(hr)

    return np.mean(hr_scores)




In [17]:
def rr_at_k(actual, predicted, k=5):
    """
    Computes Reciprocal Rank at rank k.

    Args:
        actual: the relevant item(s)
        predicted: the ranked list of items
        k: rank cutoff (default is 5)

    Returns:
        Reciprocal Rank at rank k.
    """

    for i, item in enumerate(predicted[:k]):
        if item == actual:
            return 1 / (i + 1)
    return 0.0


def mrr_at_k(actual, predicted, k=5):
    """
    Computes Mean Reciprocal Rank (MRR) at rank k.

    Args:
        actual: list of relevant items (can be multiple)
        predicted: ranked list of items
        k: rank cutoff (default is 5)

    Returns:
        MRR at rank k.
    """

    if actual is None or len(predicted) == 0:
        return 0.0

    rr_scores = []
    for item in actual:
        rr = rr_at_k(item, predicted, k)
        rr_scores.append(rr)

    return np.mean(rr_scores)



# Definition fairness

In [18]:



def calculate_disparate_impact(protected_outcomes, privileged_outcomes):
    """
    Args:
        protected_outcomes: List of binary outcomes (1=favorable) for the protected group.
        privileged_outcomes: List of binary outcomes for the privileged group.
    Returns:
        Disparate impact ratio.
    """
    protected_rate = np.mean(protected_outcomes)
    privileged_rate = np.mean(privileged_outcomes)

    if privileged_rate == 0:
        return np.inf

    return protected_rate / privileged_rate


def calculate_group_recommender_unfairness(group1_metrics, group2_metrics):
  """
    Calculates the absolute difference in mean metrics between two groups.
    This metric quantifies the unfairness of a recommender system by examining
    the absolute difference in average performance between different user groups.

    Args:
        group1_metrics (list or numpy.ndarray): A list or numpy array of metrics for group 1.
        group2_metrics (list or numpy.ndarray): A list or numpy array of metrics for group 2.

    Returns:
        float: The absolute difference in mean metrics between the two groups.
  """
  return np.abs(np.mean(group1_metrics) - np.mean(group2_metrics))




In [19]:
def coefficient_of_variation(arr):
    mean_val = np.mean(arr)
    if mean_val == 0:
        return 0
    return np.std(arr) / mean_val

def coef_variation(arr):
    mean_val = np.mean(arr)
    if mean_val == 0:
        return 0
    return np.std(arr) / mean_val

def calculate_ucv(metric_low_group, metric_high_group):
    cv_low = coefficient_of_variation(metric_low_group) if len(metric_low_group) > 0 else 0
    cv_high = coefficient_of_variation(metric_high_group) if len(metric_high_group) > 0 else 0
    return (cv_low + cv_high) / 2

def coefficient_of_variance(group):
    """Calculates the coefficient of variance for a group."""
    return np.std(group) / np.mean(group) if np.mean(group) != 0 else 0




# Content-based

In [20]:
df_modcloth_clean.head()

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,...,user_id,shoe size,shoe width,review_summary,review_text,num_user_reviews,num_user_reviews_normalized,quality_normalized,interaction_strength,user_activity
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,...,991571,NaN,NaN,NaN,NaN,2,0.08,1.0,2.69,High
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,...,944840,NaN,NaN,NaN,NaN,1,0.04,1.0,2.36,Low
5,123373,27.0,11,5.0,c,41.0,36.0,new,NaN,5ft 4in,...,162012,NaN,NaN,NaN,NaN,1,0.04,1.0,2.36,Low
8,123373,NaN,30,4.0,d,50.0,42.0,new,NaN,5ft 10in,...,279568,11.0,wide,NaN,NaN,4,0.16,0.8,3.01,High
9,123373,NaN,13,5.0,dd/e,41.0,36.0,new,39,5ft 6in,...,950172,9.0,NaN,NaN,NaN,2,0.08,1.0,2.36,High


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def hr_at_k(actual_items, predicted_items, k):
    hits = len(set(actual_items) & set(predicted_items))
    return 1.0 if hits > 0 else 0.0

def mrr_at_k(actual_items, predicted_items, k):
    for rank, item in enumerate(predicted_items[:k], start=1):
        if item in actual_items:
            return 1.0 / rank
    return 0.0

def create_weighted_user_profile(user_id, df, interaction_data):
    user_items = interaction_data.loc[user_id, interaction_data.loc[user_id] > 0].index
    user_items = user_items.tolist()
    user_items_df = df[df['item_id'].isin(user_items)]
    weighted_categories = user_items_df.groupby('category')['interaction_strength'].sum()

    user_profile = ' '.join([f'{category} ' * int(weight) for category, weight in weighted_categories.items()])

    return user_profile

def recommend_items_for_user(user_id, df, interaction_data, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)

    # Vectorize the user's profile and the item categories
    vectorizer = TfidfVectorizer(stop_words='english')
    all_profiles = df['category'].tolist() + [user_profile]
    profile_matrix = vectorizer.fit_transform(all_profiles)

    cosine_sim = cosine_similarity(profile_matrix[-1:], profile_matrix[:-1]) # user items


    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get item indices and their corresponding similarity scores
    item_indices = [i[0] for i in sim_scores[:top_n]]
    item_scores = [i[1] for i in sim_scores[:top_n]]


    recommended_items = df.iloc[item_indices]

    return recommended_items[['item_id', 'category']], item_scores


recommended_items, scores = recommend_items_for_user(user_id=279568, df=df_modcloth_clean, interaction_data=train_user_item_matrix)
print("Recommended Items for User 123 based on Category and Interaction Strength:")
print(recommended_items)

Recommended Items for User 123 based on Category and Interaction Strength:
       item_id category
38581   422651     tops
38583   422651     tops
38584   422651     tops
38585   422651     tops
38587   422651     tops


In [23]:
def compute_user_similarity(target_user_interactions, interaction_data):
    similarities = cosine_similarity(target_user_interactions.values.reshape(1, -1), interaction_data.values)
    similarity_scores = pd.Series(similarities.flatten(), index=interaction_data.index)

    return similarity_scores

def create_user_based_profile(user_id, interaction_data, k=5):
    user_interactions = interaction_data.loc[user_id]
    user_similarities = compute_user_similarity(user_interactions, interaction_data)
    similar_users = user_similarities.nlargest(k).index
    recommended_items = interaction_data.loc[similar_users].mean(axis=0).sort_values(ascending=False).index
    recommended_items = [item for item in recommended_items if user_interactions[item] == 0]

    return recommended_items[:k]

In [24]:

from sklearn.metrics.pairwise import cosine_similarity

def calculate_metrics_for_user(user_id, actual_item, recommended_items, user_engaged, k=5):
    # HR@K
    hr = hr_at_k([actual_item], recommended_items, k)
    # MRR@K
    mrr = mrr_at_k([actual_item], recommended_items, k)
    # NDCG@K
    relevance_scores = [1 if item == actual_item else 0 for item in recommended_items[:k]]
    dcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(relevance_scores)])
    idcg = sum([1 / np.log2(i + 2) for i in range(min(k, len(relevance_scores)))])
    ndcg = dcg / idcg if idcg > 0 else 0
    cv = coefficient_of_variance(recommended_items)
    cv_rel = coefficient_of_variance(relevance_scores)

    return {
        'User': user_id,
        'NDCG@K': ndcg,
        'HR@K': hr,
        'MRR@K': mrr,
        'CV': cv,
        'CV_rel': cv_rel,
        'Engagement Group': user_engaged
    }


# multiple rounds

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import torch

def calculate_all_metrics_torch(data, train_user_item_matrix, device='cuda', top_n=5):
    metrics_content_based = []

    # Step 1: Ensure the user activity is available in the data (Low or High engagement)
    data['user_engaged'] = data['review_summary'].notnull()  # Assuming a user engagement column is already present

    test_items = data.groupby("user_id")["item_id"].first().to_dict()

    # Step 2: Loop through each user
    for user_id in data['user_id'].unique():
        actual_item = test_items.get(user_id, None)
        if actual_item is None:
            continue

        user_data = data[data['user_id'] == user_id]

        # Mapping user activity based on the 'user_activity' column
        user_activity = user_data['user_activity'].iloc[0]  # Assuming 'user_activity' is available

        # Divide into 'Low' and 'High' based on user activity
        if user_activity == 'Low':
            user_engaged = "Low"
        elif user_activity == 'Moderate/High':
            user_engaged = "High"
        else:
            user_engaged = "Unknown"  # In case there are any unexpected values in 'user_activity'
        # merge unknown and high
        if user_engaged == "Unknown":
            user_engaged = "High"

        # Step 3: Create a content-based recommendation system
        recommended_items = recommend_items_for_user_content_based(user_id, data, train_user_item_matrix, top_n=top_n)

        # Step 4: Calculate HR, MRR, NDCG for the user
        metrics_content_based.append(calculate_metrics_for_user(user_id, actual_item, recommended_items, user_engaged))

    # Step 5: Convert the results into a DataFrame for aggregation
    metrics_content_based_df = pd.DataFrame(metrics_content_based, columns=['User', 'NDCG@K', 'HR@K', 'MRR@K','CV','CV_rel', 'Engagement Group'])

    # Step 6: Group by Engagement Level and Calculate the Mean for each metric
    grouped_metrics_content_based = metrics_content_based_df.groupby('Engagement Group').agg({
        'NDCG@K': 'mean',
        'HR@K': 'mean',
        'MRR@K': 'mean',
        'CV': 'mean',
        'CV_rel': 'mean'

    })

    return grouped_metrics_content_based

def recommend_items_for_user_content_based(user_id, data, train_user_item_matrix, top_n=5):
    """
    Content-based recommendation logic: Recommend items based on item content and user's interaction.
    """
    # Step 1: Create the user's profile (this assumes you have a function to create a weighted user profile)
    user_profile = create_weighted_user_profile(user_id, data, train_user_item_matrix)

    # Step 2: Vectorize the item content (e.g., item category, description, etc.)
    vectorizer = TfidfVectorizer(stop_words='english')
    all_profiles = data['category'].tolist() + [user_profile]  # Assuming 'category' is a feature
    profile_matrix = vectorizer.fit_transform(all_profiles)

    # Step 3: Compute cosine similarity between the user profile and all item profiles
    cosine_sim = cosine_similarity(profile_matrix[-1:], profile_matrix[:-1])

    # Step 4: Get the top_n most similar items
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Step 5: Extract the top N recommended items
    recommended_items = [data.iloc[i[0]]['item_id'] for i in sim_scores[:top_n]]

    return recommended_items

def create_weighted_user_profile(user_id, data, interaction_data):
    """
    Create a weighted profile for the user based on their interactions with items.
    This profile will be used for content-based recommendations.
    """
    if user_id not in interaction_data.index:
        return ' '
    user_items = interaction_data.loc[user_id, pd.to_numeric(interaction_data.loc[user_id], errors='coerce') > 0].index

    # Convert item IDs to a list
    user_items = user_items.tolist()

    # Get the categories and interaction strengths for those items
    user_items_df = data[data['item_id'].isin(user_items)]  # Select rows from data where item_id is in user_items list
    weighted_categories = user_items_df.groupby('category')['interaction_strength'].sum()

    # Create a "profile" that is a combination of weighted categories
    user_profile = ' '.join([f'{category} ' * int(weight) for category, weight in weighted_categories.items()])

    return user_profile

# Assuming 'train_user_item_matrix' is a pandas DataFrame where rows represent users, and columns represent items
# Each entry in the DataFrame is the interaction strength or rating of the user-item pair
# Execute the function
device = 'cuda' if torch.cuda.is_available() else 'cpu'
grouped_metrics_content_based = calculate_all_metrics_torch(df_modcloth_clean, train_user_item_matrix, device)

#grouped_metrics_content_based
print(grouped_metrics_content_based)
# Display the metrics
#import ace_tools as tools; tools.display_dataframe_to_user(name="Content-Based Metrics", dataframe=grouped_metrics_content_based)


KeyboardInterrupt: 

In [ ]:

# Extract NDCG values for each group
ndcg_low = grouped_metrics_optimized_sample_gpu.loc["Low", "NDCG@K"]
ndcg_moderate_high = grouped_metrics_optimized_sample_gpu.loc["High", "NDCG@K"]

# Calculate metrics for NDCG
di_ndcg = calculate_disparate_impact(ndcg_low, ndcg_moderate_high)
gru_ndcg = calculate_group_recommender_unfairness(ndcg_low, ndcg_moderate_high)
cv_low = coefficient_of_variance(grouped_metrics_optimized_sample_gpu.loc["Low", "NDCG@K"])
cv_moderate_high = coefficient_of_variance(grouped_metrics_optimized_sample_gpu.loc["High", "NDCG@K"])



print(f"\nLightFM - NDCG Metrics:")
print(f"Disparate Impact: {di_ndcg}")
print(f"Group Recommender Unfairness: {gru_ndcg}")
print(f"Coefficient of Variance (Low): {cv_low}")
print(f"Coefficient of Variance (High): {cv_moderate_high}")



In [ ]:
import torch
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import TruncatedSVD
import pandas as pd
from scipy import sparse

# Step 1: Create the weighted user profile (same as before)
def create_weighted_user_profile(user_id, df, interaction_data):
    if user_id not in interaction_data.index:
        return ' '
    user_items = interaction_data.loc[user_id, pd.to_numeric(interaction_data.loc[user_id], errors='coerce') > 0].index
    user_items = user_items.tolist()
    user_items_df = df[df['item_id'].isin(user_items)]
    weighted_categories = user_items_df.groupby('category')['interaction_strength'].sum()
    user_profile = ' '.join([f'{category} ' * int(weight) for category, weight in weighted_categories.items()])
    return user_profile


# Step 3: Recommend items for a user (same as before)
def recommend_items_for_user_torch(user_id, df, interaction_data, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)
    vectorizer = TfidfVectorizer(stop_words='english')
    all_profiles = df['category'].tolist() + [user_profile]
    profile_matrix = vectorizer.fit_transform(all_profiles)
    cosine_sim = cosine_similarity(profile_matrix[-1:], profile_matrix[:-1])
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    item_indices = [i[0] for i in sim_scores[:top_n]]
    recommended_items = df.iloc[item_indices][['item_id', 'category']]
    return recommended_items['item_id'].tolist()

# Step 4: Calculate metrics for a user (same as before)
def calculate_metrics_for_user(user_id, actual_item, recommended_items, user_engaged, k=5):
    # HR@K (Hit Ratio)
    #print(f"User: {user_id}, Actual Item: {actual_item}, Recommended Items: {recommended_items}, User Engaged: {user_engaged}")
    hr = hr_at_k([actual_item], recommended_items, k)
    # MRR@K (Mean Reciprocal Rank) - Rank of the first relevant item
    mrr = mrr_at_k([actual_item], recommended_items, k)

    # NDCG@K (Normalized Discounted Cumulative Gain)
    # Relevance is binary, 1 if relevant (actual_item), 0 otherwise
    relevance_scores = [1 if item == actual_item else 0 for item in recommended_items[:k]]
    dcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(relevance_scores)])
    idcg = sum([1 / np.log2(i + 2) for i in range(min(k, len(relevance_scores)))])
    ndcg = dcg / idcg if idcg > 0 else 0
    cv = coefficient_of_variance(recommended_items)
    cv_rel = coefficient_of_variance(relevance_scores)
    mean_cv = np.mean(cv)
    mean_cv_rel = np.mean(cv_rel)
    mean_hr = np.mean(hr)
    mean_mrr = np.mean(mrr)
    mean_ndcg = np.mean(ndcg)



    print(f"UCV@K for NDCG in round {round_num + 1}: {ucv_ndcg:.4f}")

    return {
        'User': user_id,
        'NDCG@K': mean_ndcg,
        'HR@K': mean_hr ,
        'MRR@K': mean_mrr,
        'CV': mean_cv,
        'CV_rel': mean_cv_rel,
        'Engagement Group': user_engaged
    }




                   NDCG@K      HR@K     MRR@K        CV    CV_rel
Engagement Group                                                  

Low               0.003114  0.003114  0.003114  0.000352  0.000000
Unknown           0.004570  0.004624  0.004624  0.000349  0.000149



In [ ]:
for round_num, round_metrics in enumerate(grouped_metrics_all_rounds):
    print(f"Metrics for round {round_num + 1}: {round_metrics}")




Metrics for round 1: NDCG@K
Metrics for round 2: HR@K
Metrics for round 3: MRR@K
Metrics for round 4: CV
Metrics for round 5: CV_rel


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assuming hr_at_k, mrr_at_k, coefficient_of_variance are defined elsewhere

def create_weighted_user_profile(user_id, df, interaction_data):
    if user_id not in interaction_data.index:
        return ''
    # Select items user interacted with positively
    user_interactions = interaction_data.loc[user_id]
    positive_items = user_interactions[user_interactions > 0].index.tolist()
    user_items_df = df[df['item_id'].isin(positive_items)]
    weighted_categories = user_items_df.groupby('category')['interaction_strength'].sum()
    # Repeat category names proportional to interaction strength (rounded)
    user_profile = ' '.join([f"{category} " * int(round(weight)) for category, weight in weighted_categories.items()])
    return user_profile.strip()

def recommend_items_for_user_torch(user_id, df, interaction_data, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)
    vectorizer = TfidfVectorizer(stop_words='english')
    all_categories = df['category'].tolist() + [user_profile]
    tfidf_matrix = vectorizer.fit_transform(all_categories)
    user_vec = tfidf_matrix[-1]
    item_vecs = tfidf_matrix[:-1]
    cosine_sim = cosine_similarity(user_vec, item_vecs).flatten()
    top_indices = cosine_sim.argsort()[::-1][:top_n]
    recommended_items = df.iloc[top_indices]['item_id'].tolist()
    return recommended_items

def calculate_metrics_for_user(user_id, actual_item, recommended_items, engagement_group, k=5):
    hr = hr_at_k([actual_item], recommended_items, k)
    mrr = mrr_at_k([actual_item], recommended_items, k)
    relevance = [1 if item == actual_item else 0 for item in recommended_items[:k]]
    dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevance))
    idcg = sum(1 / np.log2(idx + 2) for idx in range(min(k, len(relevance))))
    ndcg = dcg / idcg if idcg > 0 else 0
    cv = coefficient_of_variance(relevance)
    return {
        'User': user_id,
        'NDCG@K': ndcg,
        'HR@K': hr,
        'MRR@K': mrr,
        'CV': cv,
        'Engagement Group': engagement_group
    }




In [ ]:
def update_user_profile(user_id, accepted_item, train_matrix, item_metadata=None):
    """
    Updates a user's profile (train_matrix) by incrementing interaction with the accepted item.

    Parameters:
        user_id (int or str): ID of the user.
        accepted_item (int or str): ID of the accepted item.
        train_matrix (pd.DataFrame): User-item interaction matrix.
        item_metadata (pd.DataFrame, optional): DataFrame with item metadata, including 'item_id' and 'category'.
    """

    # Optionally log or use item category
    if item_metadata is not None:
        row = item_metadata[item_metadata['item_id'] == accepted_item]
        if not row.empty:
            item_category = row['category'].iloc[0]


    # Ensure user exists in train_matrix
    if user_id not in train_matrix.index:
        train_matrix.loc[user_id] = 0  # Add new user row with zeros

    # Ensure item exists in train_matrix
    if accepted_item not in train_matrix.columns:
        train_matrix[accepted_item] = 0  # Add new item column with zeros

    # Increment interaction strength
    train_matrix.loc[user_id, accepted_item] += 1

    return train_matrix

In [ ]:

test_df['user_profile'] = test_df['user_id'].apply(lambda user_id: create_weighted_user_profile(user_id, df_modcloth_clean, train_user_item_matrix))



In [ ]:
def coef_variation(arr):
    mean_val = np.mean(arr)
    if mean_val == 0:
        return 0
    return np.std(arr) / mean_val

def run_rounds(df, train_matrix, rounds=3):
    """Runs multiple rounds of recommendations and updates user profiles."""

    all_round_metrics = []

    for round_num in range(rounds):
        print(f"\n--- Round {round_num + 1} ---")
        round_metrics = []

        # 1. Get test items for each user
        test_items = df.groupby("user_id")["item_id"].first().to_dict()


        for user_id, actual_item in test_items.items():
            if user_id not in train_matrix.index:
                continue  # skip users not in training matrix

            # User engagement from round_data (same window)
            user_engagement = df[df["user_id"] == user_id]["user_activity"].iloc[0]

            # Generate recommendations using only past data
            recommended_items = recommend_items_for_user_torch(user_id, df, train_matrix)

            # Calculate metrics
            metrics = calculate_metrics_for_user(user_id, actual_item, recommended_items, user_engagement)
            round_metrics.append(metrics)

            if round_num < rounds :# -1
                train_matrix = update_user_profile(user_id, actual_item, train_matrix, df)

        # Convert metrics to DataFrame for this round
        round_metrics_df = pd.DataFrame(round_metrics)

        # Compute UCV metrics
        def get_ucv(metric_name):
            low = round_metrics_df[round_metrics_df['Engagement Group'] == 'Low'][metric_name].values
            high = round_metrics_df[round_metrics_df['Engagement Group'] != 'Low'][metric_name].values
            return (coef_variation(low) + coef_variation(high)) / 2

        ucv_ndcg = get_ucv('NDCG@K')
        ucv_hr = get_ucv('HR@K')
        ucv_mrr = get_ucv('MRR@K')
        ucv_cv = get_ucv('CV')
        avg_ndcg = round_metrics_df['NDCG@K'].mean()
        avg_hr = round_metrics_df['HR@K'].mean()
        avg_mrr = round_metrics_df['MRR@K'].mean()
        print(f"Average NDCG: {avg_ndcg}")
        print(f"Average HR: {avg_hr}")
        print(f"Average MRR: {avg_mrr}")

        print(f"UCV@K for NDCG in round {round_num + 1}: {ucv_ndcg:.6f}")
        print(f"UCV@K for HR in round {round_num + 1}: {ucv_hr:.6f}")
        print(f"UCV@K for MRR in round {round_num + 1}: {ucv_mrr:.6f}")

        # Aggregate metrics by engagement group
        grouped_metrics = round_metrics_df.groupby("Engagement Group").agg(
            {"NDCG@K": "mean", "HR@K": "mean", "MRR@K": "mean", "CV": "mean"}
        )
        grouped_metrics['UCV_NDCG'] = ucv_ndcg
        all_round_metrics.append(grouped_metrics)

        print(grouped_metrics)

    return all_round_metrics

# Run the function (Example)
all_round_metrics = run_rounds(df_modcloth_clean, train_user_item_matrix)
print(f"Overall Results: {all_round_metrics}")



--- Round 1 ---


In [ ]:


def calculate_di_gru(all_round_metrics):
    """
    Calculates Disparate Impact and Group Recommender Unfairness for NDCG and HR
    for each round of metrics.

    Args:
        all_round_metrics (list): A list of DataFrames, where each DataFrame contains
                                   metrics for a single round, grouped by Engagement Group.
                                   Each DataFrame is expected to have 'NDCG@K' and 'HR@K' columns
                                   and 'Low' and 'High' index labels.
    """
    for round_num, round_metrics_df in enumerate(all_round_metrics):
        print(f"\n--- Fairness Metrics for Round {round_num + 1} ---")

        # Ensure both groups exist in the DataFrame
        if 'Low' in round_metrics_df.index and 'High' in round_metrics_df.index:
            # Extract metrics for 'Low' (protected) and 'High' (privileged) groups
            ndcg_low = round_metrics_df.loc["Low", "NDCG@K"]
            ndcg_high = round_metrics_df.loc["High", "NDCG@K"]
            hr_low = round_metrics_df.loc["Low", "HR@K"]
            hr_high = round_metrics_df.loc["High", "HR@K"]
            mrr_low = round_metrics_df.loc["Low", "MRR@K"]
            mrr_high = round_metrics_df.loc["High", "MRR@K"]

            # Calculate Disparate Impact (DI)
            di_ndcg = calculate_disparate_impact([ndcg_low], [ndcg_high])
            di_hr = calculate_disparate_impact([hr_low], [hr_high])
            di_mrr = calculate_disparate_impact([mrr_low], [mrr_high])

            # Calculate Group Recommender Unfairness (GRU)
            gru_ndcg = calculate_group_recommender_unfairness([ndcg_low], [ndcg_high])
            gru_hr = calculate_group_recommender_unfairness([hr_low], [hr_high])
            gru_mrr = calculate_group_recommender_unfairness([mrr_low], [mrr_high])


            print(f"NDCG:")
            print(f"  Disparate Impact (Low/High): {di_ndcg:.4f}")
            print(f"  Group Recommender Unfairness (Abs Diff): {gru_ndcg:.4f}")
            print(f"MRR:")
            print(f"  Disparate Impact (Low/High): {di_mrr:.4f}")
            print(f"  Group Recommender Unfairness (Abs Diff): {gru_mrr:.4f}")

            print(f"HR@K:")
            print(f"  Disparate Impact (Low/High): {di_hr:.4f}")
            print(f"  Group Recommender Unfairness (Abs Diff): {gru_hr:.4f}")
        else:
            print("Could not calculate fairness metrics as one or both engagement groups are missing.")


# Calculate DI and GRU for all rounds
calculate_di_gru(all_round_metrics)


--- Fairness Metrics for Round 1 ---
NDCG:
  Disparate Impact (Low/High): 2.6509
  Group Recommender Unfairness (Abs Diff): 0.0181
MRR:
  Disparate Impact (Low/High): 2.5490
  Group Recommender Unfairness (Abs Diff): 0.0178
HR@K:
  Disparate Impact (Low/High): 2.4479
  Group Recommender Unfairness (Abs Diff): 0.0176

--- Fairness Metrics for Round 2 ---
NDCG:
  Disparate Impact (Low/High): 2.5014
  Group Recommender Unfairness (Abs Diff): 0.0179
MRR:
  Disparate Impact (Low/High): 2.3965
  Group Recommender Unfairness (Abs Diff): 0.0175
HR@K:
  Disparate Impact (Low/High): 2.2843
  Group Recommender Unfairness (Abs Diff): 0.0172

--- Fairness Metrics for Round 3 ---
NDCG:
  Disparate Impact (Low/High): 2.5014
  Group Recommender Unfairness (Abs Diff): 0.0179
MRR:
  Disparate Impact (Low/High): 2.3965
  Group Recommender Unfairness (Abs Diff): 0.0175
HR@K:
  Disparate Impact (Low/High): 2.2843
  Group Recommender Unfairness (Abs Diff): 0.0172


In [ ]:
def update_user_profile(user_id, actual_item, train_matrix):
    """Example of updating user profile based on interaction."""

    # This is a simplified example. You might update the user's
    # interaction strength or add new item features to the profile.
    if user_id in train_matrix.index and actual_item in train_matrix.columns:
        current_interaction = train_matrix.loc[user_id, actual_item]
        # Update interaction strength (example)
        train_matrix.loc[user_id, actual_item] = current_interaction + 1 # increment

# New section

--- Round 1 ---
                    NDCG@K      HR@K     MRR@K        CV
Engagement Group                                        
High              0.010867  0.012047  0.011402  0.003007
Low               0.027763  0.028419  0.028008  0.001568

--- Round 2 ---
                    NDCG@K      HR@K     MRR@K        CV
Engagement Group                                        
High              0.011797  0.013264  0.012436  0.003738
Low               0.028439  0.029198  0.028722  0.001827

--- Round 3 ---
                    NDCG@K      HR@K     MRR@K        CV
Engagement Group                                        
High              0.011797  0.013264  0.012436  0.003738
Low               0.028439  0.029198  0.028722  0.001827
Overall Results: [                    NDCG@K      HR@K     MRR@K        CV
Engagement Group                                        
High              0.010867  0.012047  0.011402  0.003007
Low               0.027763  0.028419  0.028008  0.001568,                     NDCG@K      HR@K     MRR@K        CV
Engagement Group                                        
High              0.011797  0.013264  0.012436  0.003738
Low               0.028439  0.029198  0.028722  0.001827,                     NDCG@K      HR@K     MRR@K        CV
Engagement Group                                        
High              0.011797  0.013264  0.012436  0.003738
Low               0.028439  0.029198  0.028722  0.001827]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def create_weighted_user_profile(user_id, df, interaction_data):
    if user_id not in interaction_data.index:
        return ' '
    user_items = interaction_data.loc[user_id, pd.to_numeric(interaction_data.loc[user_id], errors='coerce') > 0].index.tolist()
    user_items_df = df[df['item_id'].isin(user_items)]
    weighted_categories = user_items_df.groupby('category')['interaction_strength'].sum()
    user_profile = ' '.join([f'{category} ' * int(weight) for category, weight in weighted_categories.items()])
    return user_profile


def recommend_items_for_user(user_id, df, interaction_data, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)


    vectorizer = TfidfVectorizer(stop_words='english')
    all_profiles = df['category'].tolist() + [user_profile]
    profile_matrix = vectorizer.fit_transform(all_profiles)

    cosine_sim = cosine_similarity(profile_matrix[-1:], profile_matrix[:-1])

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    item_indices = [i[0] for i in sim_scores[:top_n]]
    item_scores = [i[1] for i in sim_scores[:top_n]]

    recommended_items = df.iloc[item_indices]

    return recommended_items[['item_id', 'category']], item_scores


#hybrid

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
def recommend_items_for_user(user_id, df, interaction_data, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)

    vectorizer = TfidfVectorizer(stop_words='english')
    all_profiles = df['category'].tolist() + [user_profile]
    profile_matrix = vectorizer.fit_transform(all_profiles)

    cosine_sim = cosine_similarity(profile_matrix[-1:], profile_matrix[:-1])

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    item_indices = [i[0] for i in sim_scores[:top_n]]
    item_scores = [i[1] for i in sim_scores[:top_n]]


    recommended_items_ids = df.iloc[item_indices]['item_id'].tolist()

    return recommended_items_ids, item_scores # witj item_scores for hyperparameter tuning

def calculate_metrics_content_based(user_id, actual_item, recommended_items, item_scores, user_engaged, k=5):
    """Calculates evaluation metrics for content-based recommendations."""
    hr = hr_at_k([actual_item], recommended_items, k)
    mrr = mrr_at_k([actual_item], recommended_items, k)
    relevance_scores = [1 if item == actual_item else 0 for item in recommended_items[:k]]

    ndcg = ndcg_at_k(relevance_scores, k)
    scores_topk = item_scores[:k]
    if len(scores_topk) > 0 and np.mean(scores_topk) != 0:
         cv = np.std(scores_topk) / np.mean(scores_topk)
    else:
         cv = 0
    return {
        'User': user_id,
        'NDCG@K': ndcg,
        'HR@K': hr,
        'MRR@K': mrr,
        'CV': cv,
        'Engagement Group': user_engaged
    }
test_items = df_modcloth_clean.groupby("user_id")["item_id"].first().to_dict()
def hybrid_recommendations(user_id, df, train_user_item_matrix, nn_model, content_weight, collaborative_weight, top_k=5):
    """
    Generate hybrid recommendations by combining content-based and collaborative filtering.

    Args:
        user_id: The ID of the user for whom to generate recommendations.
        df: The DataFrame containing item data.
        train_user_item_matrix: The user-item interaction matrix.
        nn_model: The NearestNeighbors model for collaborative filtering.
        content_weight: The weight to give to content-based recommendations.
        collaborative_weight: The weight to give to collaborative filtering recommendations.
        top_k: The number of recommendations to generate.

    Returns:
        A list of recommended item IDs and their combined scores.
    """

    content_recs_list, content_scores_raw = recommend_items_for_user(user_id, df, train_user_item_matrix, top_n=top_k)

    collaborative_recs_list = []
    collaborative_scores_raw = []
    if user_id in train_user_item_matrix.index:
        user_index = train_user_item_matrix.index.get_loc(user_id)
        if user_index < train_user_item_matrix.shape[0]:
            try:
                distances, neighbor_indices = nn_model.kneighbors(train_user_item_matrix.iloc[user_index, :].values.reshape(1, -1), n_neighbors=min(10, train_user_item_matrix.shape[0])) # Added min to avoid error with small matrix
                collaborative_recs_with_scores = get_user_based_recommendations(user_index, neighbor_indices[0], train_user_item_matrix, distances, top_k=top_k)
                collaborative_recs_list = [item_id for item_id, score in collaborative_recs_with_scores]
                collaborative_scores_raw = [score for item_id, score in collaborative_recs_with_scores]
            except ValueError as e:
                 print(f"Error during collaborative filtering for user {user_id}: {e}")

                 pass


    max_content_score = max(content_scores_raw) if content_scores_raw else 1.0
    max_collaborative_score = max(collaborative_scores_raw) if collaborative_scores_raw else 1.0


     for item_id in all_recs:
        content_score = content_scores.get(item_id, 0)
        collaborative_score = collaborative_scores.get(item_id, 0)

        normalized_content_score = (content_weight * content_score / max_content_score) if max_content_score != 0 else 0
        normalized_collaborative_score = (collaborative_weight * collaborative_score / max_collaborative_score) if max_collaborative_score != 0 else 0

        all_recs[item_id] = normalized_content_score + normalized_collaborative_scor

    sorted_recs = sorted(all_recs.items(), key=lambda x: x[1], reverse=True)[:top_k]
    recommended_item_ids = [item_id for item_id, score in sorted_recs]
    combined_scores = [score for item_id, score in sorted_recs]

    return recommended_item_ids, combined_scores




nn_model = NearestNeighbors(metric='cosine')
nn_model.fit(train_user_item_matrix)
param_grid = {
    'content_weight': [0.1, 0.3, 0.5, 0.7, 0.9],
    'collaborative_weight': [0.1, 0.3, 0.5, 0.7, 0.9],
}

best_average_ndcg = -1
best_params = {}

for params in ParameterGrid(param_grid):
    total_weight = params['content_weight'] + params['collaborative_weight']
    if total_weight == 0:
        continue
    normalized_content_weight = params['content_weight'] / total_weight
    normalized_collaborative_weight = params['collaborative_weight'] / total_weight

    metrics_list = []


    unique_user_ids = df_modcloth_clean['user_id'].unique()

    for user_id in unique_user_ids:
        if user_id not in train_user_item_matrix.index:
            continue

        actual_item = test_items.get(user_id, None)

        if actual_item is None:
            continue

        user_data = df_modcloth_clean[df_modcloth_clean['user_id'] == user_id]
        if user_data.empty:
            continue

        user_activity = user_data['user_activity'].iloc[0]

        if user_activity == 'Low':
            user_engaged = "Low"
        else:
            user_engaged = "High"

        recommended_items, recommended_scores = hybrid_recommendations(
            user_id,
            df_modcloth_clean,
            train_user_item_matrix,
            nn_model,
            content_weight=normalized_content_weight,
            collaborative_weight=normalized_collaborative_weight,
            top_k=5 # in future hyperparameter for k
        )

        if recommended_items:
            metrics = calculate_metrics_for_user(
                user_id,
                actual_item,
                recommended_items,
                user_engaged,
                k=5
            )
            metrics_list.append(metrics)


    if metrics_list:
        metrics_df = pd.DataFrame(metrics_list)
        average_ndcg_for_params = metrics_df['NDCG@K'].mean()


        if average_ndcg_for_params > best_average_ndcg:
            best_average_ndcg = average_ndcg_for_params
            best_params = params
    else:
        print(f"No metrics calculated for params: {params}")

print(f"Best Hyperparameters: {best_params}")
print(f"Best Average NDCG: {best_average_ndcg}")


Streaming output truncated to the last 5000 lines.
<ipython-input-36-cd41e283509d>:92: RuntimeWarning: invalid value encountered in scalar divide
  all_recs[item_id] = (content_weight * content_score / max_content_score) + (collaborative_weight * collaborative_score / max_collaborative_score)
<ipython-input-36-cd41e283509d>:92: RuntimeWarning: invalid value encountered in scalar divide
  all_recs[item_id] = (content_weight * content_score / max_content_score) + (collaborative_weight * collaborative_score / max_collaborative_score)
<ipython-input-36-cd41e283509d>:92: RuntimeWarning: invalid value encountered in scalar divide
  all_recs[item_id] = (content_weight * content_score / max_content_score) + (collaborative_weight * collaborative_score / max_collaborative_score)
<ipython-input-36-cd41e283509d>:92: RuntimeWarning: invalid value encountered in scalar divide
  all_recs[item_id] = (content_weight * content_score / max_content_score) + (collaborative_weight * collaborative_score / ma

Hyperparameter tuning  best parameter 0.5 0.5

In [ ]:
nn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric='euclidean')
nn_model.fit(train_user_item_matrix)
distances, indices = nn_model.kneighbors(train_user_item_matrix)

In [ ]:
###############################################
# 3. Recommendation User based model
###############################################
def get_user_based_recommendations(user_index, neighbor_indices, train_matrix, distances, top_k=5):
    """
    Generate Recommendations for a user based on preference or similar users (neighbors) using user-based CF.

    Args:
        user_index: index of user in the user-item matrix
        neighbor_indices: indices of the nearest neighbors
        train_matrix: user-item matrix
        distances: distance matrix of nearest neighbors
        top_k: number of recommendations to generate

    Returns:
        recommendations: list of recommended items for the user
    """

    user_interacted = train_matrix.columns[train_matrix.iloc[user_index] > 0].tolist()
    rec_scores = {}

    for i, neighbor in enumerate(neighbor_indices[:10]):
        if neighbor == user_index:
            continue
        neighbor_vector = train_matrix.iloc[neighbor]

        for item, score in neighbor_vector.items():
            if score > 0 and item not in user_interacted:

                rec_scores[item] = rec_scores.get(item, 0) + score * (1 / (1 + distances[0][i]))
    if not rec_scores:
        return [(item, 0) for item in np.random.choice(train_matrix.columns, size=top_k, replace=False).tolist()]
    return sorted(rec_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]



In [ ]:
###############################################
# 4. Hybrid Recommender
###############################################
def hybrid_recommendations(user_id, df, train_user_item_matrix, content_weight=0.5, collaborative_weight=0.5, top_k=5):
    """
    Generate hybrid recommendations by combining content-based and collaborative filtering.

    Args:
        user_id: The ID of the user for whom to generate recommendations.
        df: The DataFrame containing item data.
        train_user_item_matrix: The user-item interaction matrix.
        content_weight: The weight to give to content-based recommendations.
        collaborative_weight: The weight to give to collaborative filtering recommendations.
        top_k: The number of recommendations to generate.

    Returns:
        A list of recommended item IDs.
    """
    content_recs, content_scores = recommend_items_for_user(user_id, df, train_user_item_matrix, top_n=top_k)
    content_recs_list = content_recs['item_id'].tolist()
    user_index = train_user_item_matrix.index.get_loc(user_id)
    distances, neighbor_indices = nn_model.kneighbors(train_user_item_matrix.iloc[user_index, :].values.reshape(1, -1), n_neighbors=10)
    collaborative_recs = get_user_based_recommendations(user_index, neighbor_indices[0], train_user_item_matrix, distances, top_k=top_k)
    collaborative_recs_list = [(item_id, score) if isinstance(item_id, int) else (item_id, score) for item_id, score in collaborative_recs]

    collaborative_recs_list = [item_id for item_id, _ in collaborative_recs_list]
    all_recs = {}
    for item_id in set(content_recs_list + collaborative_recs_list):
        content_score = next((score for rec_item_id, score in zip(content_recs['item_id'], content_scores) if rec_item_id == item_id), 0)
        collaborative_score = next((score for rec_item_id, score in collaborative_recs if rec_item_id == item_id), 0)
        all_recs[item_id] = content_weight * content_score + collaborative_weight * collaborative_score
    sorted_recs = sorted(all_recs.items(), key=lambda x: x[1], reverse=True)[:top_k]

    return sorted_recs

In [ ]:
def calculate_metrics_for_hybrid(user_id, actual_item, recommendations, engagement_level, k=5):
    """Calculates NDCG, MRR, and HR for hybrid recommendations."""
    recommended_items = [item_id for item_id, _ in recommendations]
    relevance_scores = [1 if item == actual_item else 0 for item in recommended_items]
    ndcg = ndcg_at_k(relevance_scores, k)
    hr = hr_at_k([actual_item], recommended_items, k)
    mrr = mrr_at_k([actual_item], recommended_items, k)
    cv = coefficient_of_variance(recommended_items)

    return [user_id, ndcg, hr, mrr,cv, engagement_level]


In [ ]:

print("Distances shape:", distances.shape)
print("Indices shape:", indices.shape)
user_id = 279568
recommendations = hybrid_recommendations(user_id, df_modcloth_clean, train_user_item_matrix, top_k=5)

print(f"Hybrid Recommendations for User {user_id}:")
for item_id in recommendations:
    print(item_id)

Distances shape: (15513, 5)
Indices shape: (15513, 5)
Hybrid Recommendations for User 279568:
(422651, np.float64(0.4949199863128662))
(427041, 0.0)
(144486, 0.0)
(572333, 0.0)
(436339, 0.0)


In [ ]:
def create_weighted_user_profile(user_id, df, interaction_data):
    if user_id not in interaction_data.index:
        return ' '
    user_items = interaction_data.loc[user_id, pd.to_numeric(interaction_data.loc[user_id], errors='coerce') > 0].index
    user_items = user_items.tolist()
    user_items_df = df[df['item_id'].isin(user_items)]
    weighted_categories = user_items_df.groupby('category')['interaction_strength'].sum()
    user_profile = ' '.join([f'{category} ' * int(weight) for category, weight in weighted_categories.items()])
    return user_profile


def recommend_items_for_user_torch(user_id, df, interaction_data, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)
    vectorizer = TfidfVectorizer(stop_words='english')
    all_profiles = df['category'].tolist() + [user_profile]
    profile_matrix = vectorizer.fit_transform(all_profiles)
    cosine_sim = cosine_similarity(profile_matrix[-1:], profile_matrix[:-1])
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    item_indices = [i[0] for i in sim_scores[:top_n]]
    recommended_items = df.iloc[item_indices][['item_id', 'category']]
    return recommended_items['item_id'].tolist()


In [ ]:
test_items = test_df.groupby("user_id")["item_id"].first().to_dict()
metrics_hybrid = []
for user_id in train_user_item_matrix.index:
    actual_item = test_items.get(user_id, None)
    if actual_item is None:
        continue
    engagement_level = df_modcloth_clean[df_modcloth_clean['user_id'] == user_id]['user_activity'].values
    if len(engagement_level) == 0:
        engagement_level = ["Unknown"]
    else:
        engagement_level = engagement_level[0]

    recommendations = hybrid_recommendations(user_id, df_modcloth_clean, train_user_item_matrix, top_k=5)
    metrics_hybrid.append(calculate_metrics_for_hybrid(user_id, actual_item, recommendations, engagement_level))

metrics_hybrid_df = pd.DataFrame(metrics_hybrid, columns=["User", "NDCG@K", "HR@K", "MRR@K", "CV","Engagement Group"])

grouped_metrics_hybrid = metrics_hybrid_df.groupby("Engagement Group").agg({"NDCG@K": "mean", "HR@K": "mean", "MRR@K": "mean","CV":"mean"})

print("\nHybrid Metrics by User Engagement Level:")
print(grouped_metrics_hybrid)


Hybrid Metrics by User Engagement Level:
                    NDCG@K      HR@K     MRR@K        CV
Engagement Group                                        
High              0.016993  0.029807  0.012759  0.374731
Low               0.017419  0.024980  0.014948  0.407266


In [ ]:
import torch


def calculate_all_metrics_for_rounds_hybrid(data, train_user_item_matrix, rounds=3, device='cuda'):
    all_rounds_metrics = []

    data['user_engaged'] = data['review_summary'].notnull()
    test_items = data.groupby("user_id")["item_id"].first().to_dict()

    for round_num in range(rounds):
        print(f"Starting round {round_num + 1} of hybrid recommendation...")

        metrics_hybrid = []
        for user_id in data['user_id'].unique():
            if user_id not in train_user_item_matrix.index:
                continue
            actual_item = test_items.get(user_id, None)
            if actual_item is None:
                continue

            user_data = data[data['user_id'] == user_id]
            user_activity = df_modcloth_clean.loc[df_modcloth_clean['user_id'] == user_id, 'user_activity'].iloc[0]

            if user_activity == 'Low':
                user_engaged = "Low"
            else:
                user_engaged = "High"


            recommendations = hybrid_recommendations(user_id, data, train_user_item_matrix, top_k=5)
            metrics_hybrid.append(calculate_metrics_for_hybrid(user_id, actual_item, recommendations, user_engaged))

        metrics_hybrid_df = pd.DataFrame(metrics_hybrid, columns=["User", "NDCG@K", "HR@K", "MRR@K","CV","Engagement Group"])
        grouped_metrics_hybrid = metrics_hybrid_df.groupby("Engagement Group").agg({"NDCG@K": "mean", "HR@K": "mean","CV":"mean", "MRR@K": "mean"})
        def get_ucv(metric_name):
            low = metrics_hybrid_df[metrics_hybrid_df['Engagement Group'] == 'Low'][metric_name].values
            high = metrics_hybrid_df[metrics_hybrid_df['Engagement Group'] != 'Low'][metric_name].values
            return (coef_variation(low) + coef_variation(high)) / 2

        ucv_ndcg = get_ucv('NDCG@K')
        ucv_hr = get_ucv('HR@K')
        ucv_mrr = get_ucv('MRR@K')
        ucv_cv = get_ucv('CV')

        grouped_metrics_hybrid['UCV_NDCG'] = ucv_ndcg
        grouped_metrics_hybrid['UCV_HR'] = ucv_hr
        grouped_metrics_hybrid['UCV_MRR'] = ucv_mrr
        avg_ndcg = grouped_metrics_hybrid['NDCG@K'].mean()
        avg_hr = grouped_metrics_hybrid['HR@K'].mean()
        avg_mrr = grouped_metrics_hybrid['MRR@K'].mean()
        print(f"Average NDCG: {avg_ndcg:.6f}")
        print(f"Average HR: {avg_hr:.6f}")
        print(f"Average MRR: {avg_mrr:.6f}")
        di_ndcg = calculate_disparate_impact([grouped_metrics_hybrid.loc['Low', 'NDCG@K']], [grouped_metrics_hybrid.loc['High', 'NDCG@K']])
        di_hr = calculate_disparate_impact([grouped_metrics_hybrid.loc['Low', 'HR@K']], [grouped_metrics_hybrid.loc['High', 'HR@K']])
        di_mrr = calculate_disparate_impact([grouped_metrics_hybrid.loc['Low', 'MRR@K']], [grouped_metrics_hybrid.loc['High', 'MRR@K']])

        grouped_metrics_hybrid['DI_NDCG'] = di_ndcg
        grouped_metrics_hybrid['DI_HR'] = di_hr
        grouped_metrics_hybrid['DI_MRR'] = di_mrr
        gru_ndcg = calculate_group_recommender_unfairness([grouped_metrics_hybrid.loc['Low', 'NDCG@K']], [grouped_metrics_hybrid.loc['High', 'NDCG@K']])
        gru_hr = calculate_group_recommender_unfairness([grouped_metrics_hybrid.loc['Low', 'HR@K']], [grouped_metrics_hybrid.loc['High', 'HR@K']])
        gru_mrr = calculate_group_recommender_unfairness([grouped_metrics_hybrid.loc['Low', 'MRR@K']], [grouped_metrics_hybrid.loc['High', 'MRR@K']])

        grouped_metrics_hybrid['GRU_NDCG'] = gru_ndcg
        grouped_metrics_hybrid['GRU_HR'] = gru_hr
        grouped_metrics_hybrid['GRU_MRR'] = gru_mrr
        gru_ndcg = calculate_group_recommender_unfairness([grouped_metrics_hybrid.loc['Low', 'NDCG@K']], [grouped_metrics_hybrid.loc['High', 'NDCG@K']])
        gru_hr = calculate_group_recommender_unfairness([grouped_metrics_hybrid.loc['Low', 'NDCG@K']], [grouped_metrics_hybrid.loc['High', 'NDCG@K']])
        gru_mrr = calculate_group_recommender_unfairness([grouped_metrics_hybrid.loc['Low', 'NDCG@K']], [grouped_metrics_hybrid.loc['High', 'NDCG@K']])
        all_rounds_metrics.append(grouped_metrics_hybrid)
        print(grouped_metrics_hybrid)
        print("\n")

    return all_rounds_metrics


device = 'cuda' if torch.cuda.is_available() else 'cpu'
all_rounds_hybrid_metrics = calculate_all_metrics_for_rounds_hybrid(df_modcloth_clean, train_user_item_matrix, rounds=3, device=device)

for round_num, metrics in enumerate(all_rounds_hybrid_metrics):
    print(f"\nHybrid Metrics for Round {round_num + 1}:")
metrics


Starting round 1 of hybrid recommendation...
Average NDCG: 0.006347
Average HR: 0.008608
Average MRR: 0.005599
                    NDCG@K      HR@K        CV     MRR@K   UCV_NDCG  \
Engagement Group                                                      
High              0.007089  0.009204  0.382374  0.006383  11.350571   
Low               0.005605  0.008012  0.418602  0.004814  11.350571   

                     UCV_HR    UCV_MRR   DI_NDCG     DI_HR    DI_MRR  \
Engagement Group                                                       
High              10.751383  12.077087  0.790652  0.870525  0.754161   
Low               10.751383  12.077087  0.790652  0.870525  0.754161   

                  GRU_NDCG    GRU_HR   GRU_MRR  
Engagement Group                                
High              0.001484  0.001192  0.001569  
Low               0.001484  0.001192  0.001569  


Starting round 2 of hybrid recommendation...
Average NDCG: 0.006434
Average HR: 0.008869
Average MRR: 0.005633
      

KeyboardInterrupt: 

Starting round 1 of hybrid recommendation...
Average NDCG: 0.006347
Average HR: 0.008608
Average MRR: 0.005599
                    NDCG@K      HR@K        CV     MRR@K   UCV_NDCG  \
Engagement Group                                                      
High              0.007089  0.009204  0.382374  0.006383  11.350571   
Low               0.005605  0.008012  0.418602  0.004814  11.350571   

                     UCV_HR    UCV_MRR   DI_NDCG     DI_HR    DI_MRR  \
Engagement Group                                                       
High              10.751383  12.077087  0.790652  0.870525  0.754161   
Low               10.751383  12.077087  0.790652  0.870525  0.754161   

                  GRU_NDCG    GRU_HR   GRU_MRR  
Engagement Group                                
High              0.001484  0.001192  0.001569  
Low               0.001484  0.001192  0.001569  


Starting round 2 of hybrid recommendation...
Average NDCG: 0.006434
Average HR: 0.008869
Average MRR: 0.005633
                    NDCG@K      HR@K        CV     MRR@K   UCV_NDCG  \
Engagement Group                                                      
High              0.007807  0.010676  0.382620  0.006856  11.398019   
Low               0.005061  0.007061  0.418347  0.004411  11.398019   

                     UCV_HR    UCV_MRR   DI_NDCG     DI_HR    DI_MRR  \
Engagement Group                                                       
High              10.742244  12.187573  0.648318  0.661416  0.643426   
Low               10.742244  12.187573  0.648318  0.661416  0.643426   

                  GRU_NDCG    GRU_HR   GRU_MRR  
Engagement Group                                
High              0.002745  0.003615  0.002445  
Low               0.002745  0.003615  0.002445  


Starting round 3 of hybrid recommendation...
Average NDCG: 0.006516
Average HR: 0.008847
Average MRR: 0.005741
                    NDCG@K      HR@K        CV     MRR@K   UCV_NDCG  \
Engagement Group                                                      
High              0.007384  0.009817  0.381605  0.006575  11.204627   
Low               0.005648  0.007876  0.416249  0.004907  11.204627   

                     UCV_HR    UCV_MRR   DI_NDCG     DI_HR    DI_MRR  \
Engagement Group                                                       
High              10.633234  11.894171  0.764929  0.802285  0.746223   
Low               10.633234  11.894171  0.764929  0.802285  0.746223   

                  GRU_NDCG    GRU_HR   GRU_MRR  
Engagement Group                                
High              0.001736  0.001941  0.001669  
Low               0.001736  0.001941  0.001669  